In [1]:
from hydroDL.data import Dataframe, DataModel
from hydroDL import utils
import time
from hydroDL import kPath
import os
import pickle
from hydroDL.data import Dataframe, DataModel
from hydroDL.data import transform, gageII, usgs, gridMET, ntn, GLASS
from hydroDL import kPath
import time
import os
import time
import pandas as pd
import numpy as np
import json

loading package hydroDL


In [2]:
def caseFolder(caseName):
    return os.path.join(kPath.dirVeg, 'model', 'data', caseName)

In [3]:
class DataFrameVeg(Dataframe):
    def __init__(self, caseName):
        saveFolder = caseFolder(caseName)
        self.saveFolder = saveFolder
        npz = np.load(os.path.join(saveFolder, 'data.npz'))
        self.varX = npz['varX'].tolist()
        if type(npz['varY']) is str:  # adhoc fix
            self.varY = [npz['varY']]
        else:
            self.varY = npz['varY']
        self.varXC = npz['varXC'].tolist()
        self.x = npz['x']
        self.y = npz['y']
        self.xc = npz['xc']
        self.yc = None
        info = pd.read_csv(os.path.join(saveFolder, 'info.csv'))
        self.siteIdLst = info['siteId'].tolist()
        self.t = npz['t']
        self.lat = info['lat'].values
        self.lon = info['lon'].values

    def loadSubset(self, name):
        subsetFile = os.path.join(self.saveFolder, 'subset.json')
        with open(subsetFile, 'r') as f:
            subset = json.load(f)
        indS = subset[name]
        return indS

In [5]:
dataName = 'singleDaily'
df = DataFrameVeg(dataName)

# Explore

In [13]:
df.varX

['VV',
 'VH',
 'vh_vv',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'ndvi',
 'ndwi',
 'nirv',
 'Fpar',
 'Lai',
 'pr',
 'sph',
 'srad',
 'tmmn',
 'tmmx',
 'pet',
 'etr']

In [14]:
df.varY

array(['LFMC'], dtype='<U4')

In [15]:
df.varXC

['slope',
 'dem',
 'canopyHeight',
 'sand',
 'clay',
 'silt',
 'lc1',
 'lc2',
 'lc3',
 'lc4',
 'lc5',
 'lc6',
 'lc7',
 'lc8',
 'lc9']

In [17]:
df.x.shape

(1888, 335, 20)

In [18]:
df.y.shape

(1888, 335, 1)

In [19]:
df.xc.shape

(335, 15)

In [22]:
len(df.siteIdLst)

335

In [24]:
df.t.shape

(1888,)

In [26]:
df.lat.shape

(335,)

In [27]:
df.lon.shape

(335,)

In [41]:
a = np.count_nonzero(~np.isnan(df.x))
b = np.count_nonzero(np.isnan(df.x))
a / (a + b)

0.42826943144447255

In [48]:
dm = DataModel(X=df.x, XC=df.xc, Y=df.y)

In [53]:
dm.X.shape

(1888, 335, 20)

In [54]:
dm.XC.shape

(335, 15)

In [55]:
dm.Y.shape

(1888, 335, 1)

In [65]:
dm.trans(mtdDefault='minmax')

In [69]:
dm.statX["statLn"].shape

(20, 2)

In [71]:
dm.statXC["statLn"].shape

(15, 2)

In [73]:
dm.statY["statLn"].shape

(1, 2)

In [75]:
dataTup = dm.getData()

In [78]:
len(dataTup)

4

In [80]:
dataTup[0].shape

(1888, 335, 20)

In [81]:
dataTup[1].shape

(335, 15)

In [82]:
dataTup[2].shape

(1888, 335, 1)

In [85]:
from hydroDL.master import basinFull, slurm, dataTs2Range

In [87]:
rho = 45
dataEnd, (iInd, jInd) = dataTs2Range(dataTup, rho, returnInd=True)
x, xc, y, yc = dataEnd

In [89]:
x.shape

(91, 10235, 20)

In [90]:
xc.shape

(10235, 15)

In [92]:
yc.shape

(10235, 1)

### Walking thru dataTs2Range

In [128]:
x, xc, y, yc = dataTup # get data
nt = y.shape[0] # number of time measurements

In [141]:
jL, iL = np.where(~np.isnan(y).any(axis=-1))
# jL -- rows where LFMC exists
# iL -- cols where LFMC exists

10407

In [130]:
xLst, xcLst, ycLst = list(), list(), list()
for j, i in zip(jL, iL):
    if j >= rho and j < nt - rho:
        if x is not None:
            xLst.append(x[j - rho : j + rho + 1, i, :])
        if xc is not None:
            xcLst.append(xc[i, :])
        if yc is None:                
            ycLst.append(y[j, i, :])

In [137]:
xE = np.stack(xLst, axis=0)
xE.shape

(10235, 91, 20)

In [138]:
xE = xE.swapaxes(0, 1)
xE.shape

(91, 10235, 20)

In [139]:
xcE = np.stack(xcLst, axis=0)
xcE.shape

(10235, 15)

In [140]:
ycE = np.stack(ycLst, axis=0)
ycE.shape

(10235, 1)